In [ ]:
! pip install qiskit
! pip install qiskit_aer

In [242]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from numpy import random
import numpy as np

In [243]:
def basis(index):
  basis=[]
  for i in index:
    if i == 0:
      basis.append('Z')
    else:
      basis.append('X')
  return basis

In [244]:
def prep_qubit(bits,basises,n):
  qc = QuantumCircuit(n,n)
  for ind , (bit , basis) in enumerate(zip(bits,basises)):
    if basis == 'Z':
      if bit == 0:
        pass
      elif bit == 1:
        qc.x(ind)

    elif basis == 'X':
      if bit == 0:
        qc.h(ind)
      elif bit == 1:
        qc.h(ind)
        qc.z(ind)

  return qc

In [245]:
def measure_qubit(o_basis , qc):
  for ind , basis in enumerate(o_basis):
    if basis == 'Z':
      qc.measure(ind,ind)
    elif basis == 'X':
      qc.h(ind)
      qc.measure(ind,ind)

  sim = Aer.get_backend('aer_simulator')
  result = sim.run(qc,shots=1).result()
  results = result.get_counts().keys()
  return list(results)[0][::-1]  # because qiskt writes msb first

Alice

In [246]:
def bb84_sender(n):
  alice_bits = random.randint(2,size=n)
  alice_basis_ind = random.randint(2,size=n)

  alice_basis =  basis(alice_basis_ind)
  ciper_qubits = prep_qubit(alice_bits,alice_basis,n)

  return ciper_qubits , alice_basis , alice_bits

In [247]:
ciper_qubits ,alice_basis , alice_bits= bb84_sender(12)

Eves intercept (optional)

In [248]:
def bb84_eve(n):
  eve_basis_ind = random.randint(2,size=n)
  eve_basis = basis(eve_basis_ind)
  data = measure_qubit(eve_basis,ciper_qubits)
  return data , eve_basis

In [249]:
#uncomment to include eve intercept

# eve_bits , eve_basis = bb84_eve(12)
# # making eve_bits string to array of int
# eve_recieved_bits = []
# for d in eve_bits:
#   eve_recieved_bits.append(int(d))
# eve_recieved_bits = np.array(eve_recieved_bits)

# ciper_qubits = prep_qubit(eve_recieved_bits,eve_basis,12)

Bob

In [250]:
def bb84_reciever(n):
  bob_basis_ind = random.randint(2,size=n)
  bob_basis = basis(bob_basis_ind)
  data = measure_qubit(bob_basis,ciper_qubits)
  return data , bob_basis

In [251]:
data , bob_basis = bb84_reciever(12)

In [252]:
# making data string to array of int
bob_recieved_bits = []
for d in data:
  bob_recieved_bits.append(int(d))
bob_recieved_bits = np.array(bob_recieved_bits)

Comparison and decoding

In [253]:
alice_bits

array([1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0])

In [254]:
bob_recieved_bits

array([1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0])

In [255]:
alice_basis = np.array(alice_basis)
bob_basis = np.array(bob_basis)

In [256]:
no = []
for ind ,(al,bo) in enumerate(zip(alice_basis,bob_basis)):
  if al == bo:
    no.append(ind)

In [257]:
for i in no:
  print(alice_bits[i] , bob_recieved_bits[i])

0 1
0 0
0 1
0 0
1 1
0 0
0 0
